<a href="https://colab.research.google.com/github/meerilahi/algoTrading/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Scale Before retraining
2. Scale Back after predictiion
3. Try out predicting probabilities
4. Tune Learning Rate to capture recent trends
5. Train ARMA model and naive prediction for baseline
6. Trying adding multiple stocks to capture relations
7. Implement Backtesting and paper trading using apis
8. Try using CNN and if still didn't help then proceed to RL

In [2]:
!pip install alpha_vantage

In [3]:
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
%matplotlib inline

In [4]:
API_KEY = "XM0ZE0M05HEDFVTP"
ts = TimeSeries(key = API_KEY, output_format = 'pandas')
data = ts.get_daily(symbol = 'MSFT', outputsize = 'full')[0]

In [24]:
df = data[['4. close', '5. volume']]
df = df.rename(columns = {'4. close':'tommorrow_close', '5. volume': 'volume'})
df = df[::-1]
for i in range(1,31):
    df[f'close_lag_{i}'] = df['tommorrow_close'].shift(i)
    df[f'volume_lag_{i}'] = df['volume'].shift(i)
df = df.dropna()
df = df.iloc[::,::-1]
df['true_diff'] = df['close'] - df['close_lag_1']
df['naive_diff_pred'] = df['true_diff'].shift(1)
df['true_signal'] = df['true_diff'] > 0
df['naive_signal'] = df['naive_diff_pred'] > 0
df = df.dropna()
scaler = MinMaxScaler()
df.iloc[:,0:-6] = scaler.fit_transform(df.iloc[:,0:-6])
df

,volume_lag_30,close_lag_30,volume_lag_29,close_lag_29,volume_lag_28,close_lag_28,volume_lag_27,close_lag_27,volume_lag_26,close_lag_26,...,volume_lag_2,close_lag_2,volume_lag_1,close_lag_1,volume,close,true_diff,naive_diff_pred,true_signal,naive_signal
date,,,,,,,,,,,,,,,,,,,,,
1999-12-15,0.029603,0.171106,0.028038,0.169868,0.036345,0.169315,0.049954,0.168895,0.198323,0.165315,...,0.035022,0.180080,0.113136,0.184656,77785800.0,108.44,9.75,2.07,True,True
1999-12-16,0.028038,0.169868,0.036345,0.169315,0.049954,0.168895,0.198323,0.165315,0.083790,0.162950,...,0.113136,0.184656,0.122923,0.206207,64330100.0,113.69,5.25,9.75,True,True
1999-12-17,0.036345,0.169315,0.049954,0.168895,0.198323,0.165315,0.083790,0.162950,0.049285,0.159081,...,0.122923,0.206207,0.099930,0.217811,52949400.0,115.25,1.56,5.25,True,True
1999-12-20,0.049954,0.168895,0.198323,0.165315,0.083790,0.162950,0.049285,0.159081,0.049191,0.164607,...,0.099930,0.217811,0.080483,0.221259,19716900.0,112.75,-2.50,1.56,False,True
1999-12-21,0.198323,0.165315,0.083790,0.162950,0.049285,0.159081,0.049191,0.164607,0.032222,0.163657,...,0.080483,0.221259,0.023695,0.215734,28723000.0,115.87,3.12,-2.50,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-10,0.016573,0.948056,0.050918,0.927057,0.030145,0.954997,0.030298,0.944232,0.083280,0.883800,...,0.029825,0.843792,0.027654,0.835879,32569768.0,380.16,-13.15,-3.58,False,False
2025-03-11,0.050918,0.927057,0.030145,0.954997,0.030298,0.944232,0.083280,0.883800,0.048483,0.883955,...,0.027654,0.835879,0.045658,0.806812,30380177.0,380.45,0.29,-13.15,True,False
2025-03-12,0.030145,0.954997,0.030298,0.944232,0.083280,0.883800,0.048483,0.883955,0.033883,0.874804,...,0.045658,0.806812,0.041916,0.807453,24253567.0,383.27,2.82,0.29,True,True


In [15]:
mean_absolute_error(df['target'],df['naive_pred']), mean_absolute_error(df['target'],np.zeros(len(df)))

(1.8767829921259842, 1.2571895748031499)

In [11]:
X = df.iloc[:,:-6].to_numpy()
X = X.reshape(6350,30,2)
y = df.iloc[:,-2].to_numpy()
X.shape, y.shape

((6350, 30, 2), (6350,))

In [14]:
split_index = int(len(X) * 0.8)
X_train, y_train = X[:split_index], y[:split_index]
X_test, y_test = X[split_index:], y[split_index:]
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, input_shape = (30, 2)),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer='adam',loss='mse', metrics=['mean_absolute_error'])
model.fit(train_ds.batch(32), epochs = 15,validation_data = test_ds.batch(32))

Epoch 1/15


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


159/159 ━━━━━━━━━━━━━━━━━━━━ 10s 49ms/step - loss: 2.5748 - mean_absolute_error: 0.8788 - val_loss: 25.8003 - val_mean_absolute_error: 3.7710
Epoch 2/15
159/159 ━━━━━━━━━━━━━━━━━━━━ 6s 37ms/step - loss: 2.5847 - mean_absolute_error: 0.8809 - val_loss: 25.8247 - val_mean_absolute_error: 3.7712
Epoch 3/15
159/159 ━━━━━━━━━━━━━━━━━━━━ 7s 46ms/step - loss: 2.5904 - mean_absolute_error: 0.8811 - val_loss: 25.7974 - val_mean_absolute_error: 3.7723
Epoch 4/15
159/159 ━━━━━━━━━━━━━━━━━━━━ 6s 37ms/step - loss: 2.5844 - mean_absolute_error: 0.8802 - val_loss: 25.7930 - val_mean_absolute_error: 3.7710
Epoch 5/15
159/159 ━━━━━━━━━━━━━━━━━━━━ 7s 45ms/step - loss: 2.5873 - mean_absolute_error: 0.8809 - val_loss: 25.8111 - val_mean_absolute_error: 3.7752
Epoch 6/15
159/159 ━━━━━━━━━━━━━━━━━━━━ 9s 35ms/step - loss: 2.5795 - mean_absolute_error: 0.8800 - val_loss: 25.8122 - val_mean_absolute_error: 3.7742
Epoch 7/15
159/159 ━━━━━━━━━━━━━━━━━━━━ 7s 47ms/step - loss: 2.5804 - mean_absolute_error: 0.8804 

In [16]:
model.save("model_copy.keras")
model2 = tf.keras.models.load_model("model_copy.keras")

In [17]:
y_pred = []
for i in range(len(X_test)):
    print('Example' + str(i))
    y_pred.append(model2.predict(X_test[i].reshape(1,30,2)))
    model2.fit(X_test[i].reshape(1,30,2), y_test[i].reshape(1,1), epochs = 1)

Example0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 4.1858 - mean_absolute_error: 2.0459
Example1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 145.2303 - mean_absolute_error: 12.0512
Example2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 14.2866 - mean_absolute_error: 3.7798
Example3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 115.2321 - mean_absolute_error: 10.7346
Example4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 69.3487 - mean_absolute_error: 8.3276
Example5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 35.9335 - mean_absolute_error: 5.9945
Example6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 18.7161 - mean_absolute_error: 4.3262
Example7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/s

In [25]:
y_pred = np.array(y_pred)
y_pred = y_pred.reshape(y_pred.shape[0])

In [27]:
len(y_pred)

1270

In [ ]:
backtest = pd.DataFrame({
  "today_close" : df.iloc[split_index:]['close_lag_1'],
  "tommorrow_pred" : y_pred
})
backtest['tommorrow_pred'] = backtest['tommorrow_pred'].shift(-1)
backtest['navie_pred'] = backtest['today_close'].shift(1)
backtest["signal"] = backtest["tommorrow_pred"] > backtest["today_close"]
backtest["navie_signal"] = backtest["navie_pred"] > backtest["today_close"]
backtest.dropna(inplace = True)
backtest.head()

,today_close,tommorrow_pred,navie_pred,signal,navie_signal
1,170.17,166.446518,168.07,False,False
2,158.18,163.585403,170.17,True,True
3,162.01,167.247986,158.18,True,False
4,172.79,170.043030,162.01,False,False
5,164.51,169.537415,172.79,True,True


In [ ]:
initial_balance = 10000
balance = initial_balance
shares = 0

for i in range(len(backtest) - 1):
    price = backtest["today_close"].iloc[i]

    if price > 0:  # Ensure valid price
        if backtest["navie_signal"].iloc[i] and balance > 0:  # Buy only if we have cash
            shares = balance / price
            balance = 0
        elif not backtest["navie_signal"].iloc[i] and shares > 0:  # Sell only if we have shares
            balance = shares * price
            shares = 0

# Final balance calculation
final_price = backtest["today_close"].iloc[-1]
final_balance = balance + (shares * final_price)  # Sell remaining shares if any
profit = final_balance - initial_balance

print(f"Final Balance: ${final_balance:.2f}, Profit: ${profit:.2f}")



Final Balance: $37286.16, Profit: $27286.16
